<a href="https://colab.research.google.com/github/Samra-63/AI-ML-Felowship/blob/main/Task11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **TASK10:** Data Cleaning & Preparation (Hotel Booking)
In this task, I performed a complete data cleaning and preparation process on a hotel booking dataset which I obtained from this[ Kaggle source](https://www.kaggle.com/code/aminizahra/hotel-booking-analysis/input).

The main steps I performed are:

 **File Upload:** Loaded the dataset from local system into Google Colab.

 **Missing Values Handling:**

*   Dropped company column due to excessive missing data.

*   Filled agent with "Not Provided".


*  Filled country with the most frequent value (mode).

*   Filled children with median value.

**Removed Duplicates to ensure unique records.**

**Date Parsing:** Converted reservation_status_date into proper datetime format using dayfirst=True.

**Feature Engineering:**


*  Created total_nights by summing week nights and weekend nights.

*   Created total_guests by adding adults, children, and babies.

**Data Filtering:** Removed rows where total guests were zero.

**Outlier Handling:** Capped extreme values in adr column at the 99th percentile.

**Data Type Optimization:** Converted object columns to categorical to improve performance.

**Saved** the Cleaned Dataset for future analysis or modeling.

This notebook ensures that the data is clean, consistent, and ready for further analysis or machine learning tasks.

In [2]:
from google.colab import files
import pandas as pd

# Upload the file
uploaded = files.upload()

# Load the uploaded CSV file (auto-detect filename)
import io
df = pd.read_csv(io.BytesIO(list(uploaded.values())[0]))
df.head()


Saving hotel_booking.csv to hotel_booking.csv


,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date,name,email,phone-number,credit_card
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,...,Transient,0.0,0,0,Check-Out,01/07/2015,Ernest Barnes,Ernest.Barnes31@outlook.com,669-792-1661,************4322
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,...,Transient,0.0,0,0,Check-Out,01/07/2015,Andrea Baker,Andrea_Baker94@aol.com,858-637-6955,************9157
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,...,Transient,75.0,0,0,Check-Out,02/07/2015,Rebecca Parker,Rebecca_Parker@comcast.net,652-885-2745,************3734
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,...,Transient,75.0,0,0,Check-Out,02/07/2015,Laura Murray,Laura_M@gmail.com,364-656-8427,************5677
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,...,Transient,98.0,0,1,Check-Out,03/07/2015,Linda Hines,LHines@verizon.com,713-226-5883,************5498


In [3]:
# Check for missing values
df.isnull().sum().sort_values(ascending=False)


,0
company,112593
agent,16340
country,488
children,4
arrival_date_year,0
lead_time,0
is_canceled,0
hotel,0
stays_in_weekend_nights,0
stays_in_week_nights,0


In [4]:
# Drop 'company' column
df.drop(columns=['company'], inplace=True)

# Fill missing agent with 'Not Provided'
df['agent'] = df['agent'].fillna('Not Provided').astype(str)

# Fill missing country with mode
df['country'].fillna(df['country'].mode()[0], inplace=True)

# Fill missing children with median
df['children'].fillna(df['children'].median(), inplace=True)


<ipython-input-4-56a61513d00c>:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['country'].fillna(df['country'].mode()[0], inplace=True)
<ipython-input-4-56a61513d00c>:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)'

In [5]:
# Remove duplicate rows
df.drop_duplicates(inplace=True)


In [7]:

# Convert date column to datetime format with dayfirst=True
df['reservation_status_date'] = pd.to_datetime(df['reservation_status_date'], dayfirst=True)


In [8]:
# Total stay nights
df['total_nights'] = df['stays_in_week_nights'] + df['stays_in_weekend_nights']

# Total guests
df['total_guests'] = df['adults'] + df['children'] + df['babies']


In [9]:
# Remove rows where total guests is zero
df = df[df['total_guests'] > 0]


In [10]:
# Cap outliers in 'adr' column at 99th percentile
q_high = df['adr'].quantile(0.99)
df['adr'] = df['adr'].apply(lambda x: q_high if x > q_high else x)


In [11]:
# Convert all object-type columns to category dtype
cat_cols = df.select_dtypes(include='object').columns
df[cat_cols] = df[cat_cols].astype('category')


In [15]:
# Save cleaned dataset
df.to_csv("hotel_booking_cleaned.csv", index=False)
print(" Cleaned data saved as 'hotel_booking_cleaned.csv'")


 Cleaned data saved as 'hotel_booking_cleaned.csv'


In [16]:
# Save cleaned dataset
df.to_csv("hotel_booking_cleaned.csv", index=False)
print(" Cleaned data saved as 'hotel_booking_cleaned.csv'")


 Cleaned data saved as 'hotel_booking_cleaned.csv'


In [ ]:
# Download the file
files.download("hotel_booking_cleaned.csv")